# Knowledge Graph Question Answering for Building Knowledge Graphs using a ReAct Framework

This notebook implements an agentic workflow based on the **ReAct (Reasoning and Acting)** framework to translate natural language questions into precise SPARQL queries. The core of this notebook is a multi-turn refinement process where two specialized agents collaborate to produce, critique, and improve a SPARQL query until it is deemed correct.

---

## 🤖 Methodology: The Two-Agent Loop

The agentic workflow is orchestrated by the `SparqlRefinementAgent` class and consists of an iterative loop between two Large Language Model (LLM) powered agents:

1.  **Query Writer Agent**:
    * **Role**: To generate and revise SPARQL queries.
    * **Action**: Given a natural language question and a subset (defined by `num_triples` of the graph) of the graph, it writes an initial SPARQL query. In subsequent turns, it revises the query based on feedback.

2.  **Critique Agent**:
    * **Role**: To evaluate the query's correctness.
    * **Action**: After the Writer's query is executed, the Critique agent reviews the original question, the query itself, and a summary of the execution results. It then makes a decision:
        * `FINAL`: The query is correct and successfully answers the question. The loop terminates.
        * `IMPROVE`: The query is incorrect, incomplete, or inefficient. The agent provides specific, actionable feedback.

This feedback is then passed back to the Query Writer Agent, which begins the next iteration by generating an improved query. This cycle continues until a `FINAL` decision is reached or the maximum number of iterations is exceeded.

---

## ⚙️ Execution Workflow

The notebook follows a systematic process for each building and question:

1.  **Configuration**: Configure your API keys and URL for LLM calls.
2.  **Define Helper Functions**: Helper functions for logging data and extracting a subset of the graph is defined.
3.  **Initialize Agents**: The `SparqlRefinementAgent` is instantiated.
4.  **Define a helper function for testing a single question**: `run_single_question` function is defined for initial testing.
5.  **Configure the necessary test conditions**: Key parameters such as the target building (`BUILDING_NAME`), LLM (`MODEL_NAME`), and SPARQL endpoint are set.
4.  **Process All Questions and Buildings**: The script iterates through each question in the JSON file.
    * The **ReAct loop** is triggered for the current question.
    * The final query generated by the agent is executed.
    * The ground-truth query is executed for comparison.
    * Key metrics are computed and saved. 

---

## 📊 Evaluation Metrics

To rigorously assess the correctness of the generated SPARQL query, the following metrics are calculated and logged:

* **Arity Matching F1**: Checks if the query returned the correct **number of columns**.
* **Exact Match F1**: A strict check for identical row content and column **order**, though it ignores column names.
* **Entity Set F1**: Flexibly checks if the correct **sets of values** were retrieved in each column, regardless of row structure or column order.
* **Row Matching F1**: The most robust metric. It finds the optimal column alignment and then checks for an exact, row-for-row match of the content. This is the primary indicator of a perfectly correct query.

In [ ]:
# Standard library imports
import csv
import itertools
import json
import os
import re
import time
import traceback
import uuid
import warnings
from typing import Any, Dict, List, Optional

# Third-party imports
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel, Field, ValidationError
from pyparsing import ParseException
from rdflib import BNode, Graph, Literal, URIRef
from SPARQLWrapper import JSON, SPARQLWrapper
from metrics import (
    get_arity_matching_f1,
    get_entity_set_f1,
    get_row_matching_f1,
    get_exact_match_f1
)



### 1. Assign your API key and your base url

In [ ]:

os.environ['OPENAI_API_KEY'] = "https://www.youtube.com/watch?v=dQw4w9WgXcQ" # Change this to your actual API key for whatever service you are using or set it in your environment variables
client = OpenAI(    
    api_key=os.environ.get('OPENAI_API_KEY'),
    base_url="https://api.openai.com/v1/"
)

### 2. Define Helper Functions

In [3]:


def get_kg_subset_content(original_ttl_path: str, max_triples: int) -> str:
    """
    Parses a TTL file and returns a string containing the prefixes and the first `max_triples`.
    If the graph is smaller than max_triples, it returns the full content.
    """
    full_graph = Graph()
    try:
        full_graph.parse(original_ttl_path, format="turtle")
        print(f"🔎 Original graph '{os.path.basename(original_ttl_path)}' contains {len(full_graph)} triples.")
    except Exception as e:
        print(f"❌ ERROR: Could not parse original TTL file at {original_ttl_path}. Reason: {e}")
        return ""

    # If the graph is small enough, use the whole thing
    if len(full_graph) <= max_triples:
        print(f"   -> Graph has {len(full_graph)} triples or fewer. Using full graph content for prompt.")
        return full_graph.serialize(format="turtle")

    print(f"   -> Graph is larger than {max_triples} triples. Creating a subset for the prompt...")
    subset_graph = Graph()
    # Copy namespaces to the subset graph
    for prefix, namespace in full_graph.namespace_manager.namespaces():
        subset_graph.bind(prefix, namespace)

    # Add the first `max_triples`
    for i, triple in enumerate(full_graph):
        if i >= max_triples:
            break
        subset_graph.add(triple)

    print(f"   -> ✅ Successfully created subset context with {len(subset_graph)} triples.")
    return subset_graph.serialize(format="turtle")

# --- Evaluation and Helper Functions ---

def extract_prefixes_from_ttl(ttl_path: str) -> str:
    """Dynamically extracts PREFIX declarations from a TTL file."""
    prefixes = []
    try:
        with open(ttl_path, 'r', encoding='utf-8') as f:
            for line in f:
                stripped_line = line.strip()
                if stripped_line.lower().startswith('@prefix'):
                    parts = stripped_line.split()
                    if len(parts) >= 3:
                        prefixes.append(f"PREFIX {parts[1]} {parts[2]}")
        print(f"✅ Successfully extracted {len(prefixes)} prefixes from {os.path.basename(ttl_path)}.")
        return "\n".join(prefixes) + "\n\n"
    except FileNotFoundError:
        print(f"❌ ERROR: TTL file not found at {ttl_path}.")
        return ""
    except Exception as e:
        print(f"❌ ERROR: Could not read prefixes from {ttl_path}. Reason: {e}")
        return ""


def check_if_question_exists(question_text: str, log_filename: str, model_name: str) -> bool:
    """Checks if a question has already been logged for a specific model."""
    if not os.path.exists(log_filename):
        return False
    try:
        with open(log_filename, 'r', newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                if row.get('question') == question_text and row.get('model') == model_name:
                    print(f"✅ Result for question '{question_text[:50]}...' and model '{model_name}' already exists. Skipping.")
                    return True
    except (FileNotFoundError, Exception) as e:
        print(f"Could not read log file {log_filename}. Error: {e}")
        return False
    return False

# --- CSV Logger Class ---

LOG_FIELDNAMES = [
    'query_id', 'question_number', 'source', 'question', 'model',
    'ground_truth_sparql', 'generated_sparql',
    'syntax_ok', 'returns_results', 'perfect_match',
    'gt_num_rows', 'gt_num_cols',
    'gen_num_rows', 'gen_num_cols',
    'arity_matching_f1',
    'exact_match_f1',
    'entity_set_f1',
    'row_matching_f1',
    'less_columns_flag',
    'prompt_tokens', 'completion_tokens', 'total_tokens'
]

class CsvLogger:
    """Handles writing log data to a CSV file, appending if it exists."""
    def __init__(self, filename: str, fieldnames: List[str]):
        self.filename = filename
        self.fieldnames = fieldnames
        
        file_exists = os.path.exists(self.filename)
        
        self.file = open(self.filename, 'a', newline='', encoding='utf-8')
        self.writer = csv.DictWriter(self.file, fieldnames=self.fieldnames)
        
        if not file_exists:
            self.writer.writeheader()
            print(f"📝 New log file created. Writing to {self.filename}")
        else:
            print(f"📝 Logger initialized. Appending to {self.filename}")

    def log(self, data: Dict[str, Any]):
        """Writes a single entry to the log file."""
        self.writer.writerow(data)
        self.file.flush()

    def close(self):
        """Closes the log file."""
        self.file.close()
        print(f"✅ Logger closed. Final results saved to {self.filename}")

### 3. Initialize The `SparqlRefinementAgent` 

In [ ]:

# --- Pydantic Models for the Two-Agent Workflow ---

class SparqlQuery(BaseModel):
    """Model for the Query Writer Agent's output."""
    sparql_query: str = Field(..., description="The generated or revised SPARQL query.")

class QueryCritique(BaseModel):
    """Model for the Critique Agent's structured feedback."""
    decision: str = Field(..., description="The decision, either 'IMPROVE' or 'FINAL'.")
    feedback: str = Field(..., description="Natural language feedback explaining the decision.")


# --- The Orchestrating Agent Class ---

class SparqlRefinementAgent:
    """
    An agent that orchestrates a conversation between a Query Writer and a Critique Agent
    to iteratively develop, evaluate, and log a SPARQL query.
    Can query a remote SPARQL endpoint or a local TTL file.
    """

    def __init__(self, sparql_endpoint: str, model_name: str = "openai/o4-mini", max_iterations: int = 5):
        self.sparql_endpoint_url = sparql_endpoint
        self.model_name = model_name
        self.max_iterations = max_iterations
        self.client = client
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.total_tokens = 0
        
        # --- Differentiate between remote endpoint and local file ---
        self.graph = None
        self.is_remote = sparql_endpoint.lower().startswith("http")

        if self.is_remote:
            print(f"🌐 Remote SPARQL endpoint mode activated: {self.sparql_endpoint_url}")
        else:
            print(f"🗂️ Local TTL file mode activated. Loading graph from: {self.sparql_endpoint_url}")
            if not os.path.exists(self.sparql_endpoint_url):
                 print(f"   -> ❌ ERROR: File not found at {self.sparql_endpoint_url}. Queries will fail.")
                 return
            try:
                self.graph = Graph()
                self.graph.parse(self.sparql_endpoint_url, format="turtle")
                print(f"   -> ✅ Graph loaded successfully with {len(self.graph)} triples.")
            except Exception as e:
                print(f"   -> ❌ ERROR: Failed to load or parse the TTL file: {e}")
                self.graph = None # Ensure graph is None on failure

    def _format_rdflib_results(self, qres) -> Dict[str, Any]:
        """Converts rdflib QueryResult to the same dict format as SPARQLWrapper."""
        variables = [str(v) for v in qres.vars]
        bindings = []
        for row in qres:
            binding_row = {}
            for var_name in variables:
                term = row[var_name]
                if term is None:
                    continue
                
                term_dict = {}
                if isinstance(term, URIRef):
                    term_dict = {'type': 'uri', 'value': str(term)}
                elif isinstance(term, Literal):
                    term_dict = {'type': 'literal', 'value': str(term)}
                    if term.datatype:
                        term_dict['datatype'] = str(term.datatype)
                    if term.language:
                        term_dict['xml:lang'] = term.language
                elif isinstance(term, BNode):
                    term_dict = {'type': 'bnode', 'value': str(term)}
                
                binding_row[var_name] = term_dict
            bindings.append(binding_row)
        
        return {"results": bindings, "variables": variables}

    def _run_sparql_query(self, query: str) -> Dict[str, Any]:
        """
        Executes a SPARQL query, dispatching to rdflib (local) or SPARQLWrapper (remote).
        Returns a structured dictionary of results.
        """
        print(f"\n🔎 Running SPARQL query... (first 80 chars: {query[:80].replace(chr(10), ' ')}...)")
        
        # --- NEW: Branch for local RDF file (rdflib) ---
        if not self.is_remote:
            if self.graph is None:
                return {"summary_string": "SPARQL query failed: The local RDF graph is not loaded.", "results": [], "row_count": 0, "col_count": 0, "syntax_ok": False, "error_message": "Graph not loaded."}
            
            try:
                qres = self.graph.query(query)
                formatted_results = self._format_rdflib_results(qres)
                bindings = formatted_results["results"]
                summary = f"Query executed successfully on local graph. Found {len(bindings)} results."
                if not bindings:
                    summary = "The query executed successfully on the local graph but returned no results."
                
                return {"summary_string": summary, "results": bindings, "row_count": len(bindings), "col_count": len(formatted_results["variables"]), "syntax_ok": True, "error_message": None}
            except (ParseException, Exception) as e:
                print(f"   -> SPARQL Query (local) Failed: {e}")
                error_msg = f"The query failed to parse with the following error: {str(e)}"
                return {"summary_string": error_msg, "results": [], "row_count": 0, "col_count": 0, "syntax_ok": False, "error_message": str(e)}

        # --- Original logic for remote SPARQL endpoint ---
        else:
            try:
                sparql = SPARQLWrapper(self.sparql_endpoint_url)
                sparql.setQuery(query)
                sparql.setReturnFormat(JSON)
                results_json = sparql.query().convert()
                
                bindings = results_json.get("results", {}).get("bindings", [])
                variables = results_json.get("head", {}).get("vars", [])
                
                summary = "Query executed successfully. Here are the first 10 results:\n" + json.dumps(bindings[:10], indent=2)
                if not bindings:
                    summary = "The query executed successfully but returned no results."

                return {"summary_string": summary, "results": bindings, "row_count": len(bindings), "col_count": len(variables), "syntax_ok": True, "error_message": None}
            except Exception as e:
                print(f"   -> SPARQL Query (remote) Failed: {e}")
                return {"summary_string": f"The query failed to execute with the following error: {str(e)}", "results": [], "row_count": 0, "col_count": 0, "syntax_ok": False, "error_message": str(e)}

    def _get_structured_response(self, messages: List[Dict], response_model) -> Optional[BaseModel]:
        """Generic helper to call the LLM, parse its response, and count tokens."""
        content = ""
        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                response_format={"type": "json_object"},
                temperature=0,
            )
            if response.usage:
                self.prompt_tokens += response.usage.prompt_tokens
                self.completion_tokens += response.usage.completion_tokens
                self.total_tokens += response.usage.total_tokens
            
            content = response.choices[0].message.content
            return response_model.model_validate_json(content)
        except (ValidationError, json.JSONDecodeError) as e:
            print(f"Pydantic/JSON validation failed: {e}")
            print(f"--- Failing LLM Response ---\n{content}\n-----------------------------")
            return None
        except Exception as e:
            print(f"An unexpected error occurred during LLM call: {e}")
            return None

    def refine_and_evaluate_query(self, eval_data: Dict[str, Any], logger: CsvLogger, prefixes: str, knowledge_graph_content: str) -> None:
        """
        The main orchestration loop to refine, evaluate, and log a query.
        """
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.total_tokens = 0

        nl_question = eval_data['question']
        ground_truth_sparql = eval_data.get('ground_truth_sparql')
        
        print(f"\n🚀 Starting refinement workflow for question: '{nl_question}'")

        system_prompt = (
            f"You are an expert SPARQL developer for Brick Schema and ASHRAE 223p. "
            f"Your job is to write a single, complete SPARQL query to answer the user's request. "
            f"Here is the knowledge graph you must query:\n\n"
            f"```turtle\n{knowledge_graph_content}\n```\n\n"
            f"If you are given feedback on a prior attempt, use it to revise and improve your query. "
            f"Respond ONLY with a JSON object containing the key 'sparql_query'."
        )
        
        query_writer_messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Please write a SPARQL query that answers the following question: {nl_question}"}
        ]

        final_generated_query = ""

        for i in range(self.max_iterations):
            print(f"\n--- Iteration {i + 1} ---")
            print("✍️  Calling Query Writer Agent...")
            query_response = self._get_structured_response(query_writer_messages, SparqlQuery)
            
            if not query_response or not query_response.sparql_query:
                print("❌ Query Writer failed to produce a valid query. Aborting iteration.")
                break
            
            final_generated_query = query_response.sparql_query
            print(f"   -> Query received:\n{final_generated_query}")
            query_writer_messages.append({"role": "assistant", "content": json.dumps(query_response.model_dump())})

            results_obj = self._run_sparql_query(final_generated_query)
            print(f"   -> Results Summary: {results_obj['summary_string'][:250]}...")

            print("🧐 Calling Critique Agent...")
            critique_prompt = [
                {"role": "system", "content": "You are an expert in SPARQL especially for Brick Schema and ASHRAE 223p. Your job is to review a SPARQL query and its results based on an original question. Decide if the query is correct or needs improvement. Respond with a JSON object: `{\"decision\": \"FINAL\" | \"IMPROVE\", \"feedback\": \"...your reasoning...\"}`."},
                {"role": "user", "content": f"Original Question: \"{nl_question}\"\n\nSPARQL Query Attempt:\n```sparql\n{final_generated_query}\n```\n\nExecution Results Summary:\n{results_obj['summary_string']}"}
            ]
            critique = self._get_structured_response(critique_prompt, QueryCritique)

            if not critique:
                print("❌ Critique Agent failed. Ending refinement loop.")
                break
            
            print(f"   -> Critique Decision: {critique.decision}")
            print(f"   -> Critique Feedback: {critique.feedback}")

            if critique.decision == "FINAL":
                print("\n✅ Critique Agent approved the query. Refinement complete.")
                break
            
            feedback_for_writer = f"Your last query attempt received the following feedback: '{critique.feedback}'. Please provide a new, improved query that addresses this feedback."
            query_writer_messages.append({"role": "user", "content": feedback_for_writer})
        
        if not final_generated_query:
            print("💔 Agentic workflow could not produce a final query.")
            return


        print("\n--- Final Evaluation and Logging ---")
        gen_results_obj = self._run_sparql_query(final_generated_query)
        gt_results_obj = self._run_sparql_query(ground_truth_sparql) if ground_truth_sparql else None
        
        # Initialize metrics to default values
        arity_f1, entity_set_f1, row_matching_f1, exact_match_f1 = 0.0, 0.0, 0.0, 0.0
        less_columns_flag = False
        
        # Calculate metrics only if both ground truth and generated queries are valid
        if gt_results_obj and gt_results_obj["syntax_ok"] and gen_results_obj["syntax_ok"]:
            gold_rows = gt_results_obj["results"]
            pred_rows = gen_results_obj["results"]
            
            arity_f1 = get_arity_matching_f1(gold_rows, pred_rows)
            entity_set_f1 = get_entity_set_f1(gold_rows=gold_rows, pred_rows=pred_rows)
            row_matching_f1 = get_row_matching_f1(gold_rows=gold_rows, pred_rows=pred_rows)
            exact_match_f1 = get_exact_match_f1(gold_rows=gold_rows, pred_rows=pred_rows)
            
            # Determine if the generated query returned fewer columns than the ground truth
            less_columns_flag = gen_results_obj['col_count'] < gt_results_obj['col_count']
        
        log_entry = {
            **eval_data,
            'model': self.model_name,
            'generated_sparql': final_generated_query,
            'syntax_ok': gen_results_obj['syntax_ok'],
            'returns_results': gen_results_obj['row_count'] > 0,
            'perfect_match': row_matching_f1 == 1.0, # Row Matching F1 is the best indicator for this
            'gt_num_rows': gt_results_obj['row_count'] if gt_results_obj else 0,
            'gt_num_cols': gt_results_obj['col_count'] if gt_results_obj else 0,
            'gen_num_rows': gen_results_obj['row_count'],
            'gen_num_cols': gen_results_obj['col_count'],
            'arity_matching_f1': arity_f1,
            'entity_set_f1': entity_set_f1,
            'row_matching_f1': row_matching_f1,
            'exact_match_f1': exact_match_f1,
            'less_columns_flag': less_columns_flag,
            'prompt_tokens': self.prompt_tokens,
            'completion_tokens': self.completion_tokens,
            'total_tokens': self.total_tokens
        }
        
        logger.log(log_entry)
        print(f"📊 Log entry saved for query_id: {eval_data['query_id']}")

### 4. Define a helper function for testing a single question

In [5]:

def run_single_question():
    # The script will automatically detect if the target is a file path or a URL
    SPARQL_TARGET = LOCAL_TTL_PATH if USE_LOCAL_TTL_FILE else REMOTE_ENDPOINT_URL

    BRICK_PREFIXES = extract_prefixes_from_ttl(LOCAL_TTL_PATH)

    KNOWLEDGE_GRAPH_CONTENT = get_kg_subset_content(LOCAL_TTL_PATH, max_triples= num_triples)
    print("First 1000 chars of knowledge graph content:")
    print(KNOWLEDGE_GRAPH_CONTENT[:1000])
    with open(json_file_path, 'r', encoding='utf-8') as f:
        all_data = json.load(f)
    target_building_data = all_data[0]

    if not os.path.exists(os.path.dirname(LOG_FILE)):
        os.makedirs(os.path.dirname(LOG_FILE))

    logger = CsvLogger(filename=LOG_FILE, fieldnames=LOG_FIELDNAMES)
    agent = SparqlRefinementAgent(
        sparql_endpoint=SPARQL_TARGET, 
        model_name=MODEL_NAME, 
        max_iterations=3
    )

    print(f"--- Processing building: {BUILDING_NAME} for model: {MODEL_NAME} ---")

    first_question_processed = False

    try:
        for query_info in target_building_data.get('queries', []):
            query_id = query_info.get('query_id')
            ground_truth_sparql = query_info.get('sparql_query')

            if not ground_truth_sparql:
                print(f"⚠️ Warning: Skipping Query Group ID {query_id} (no ground truth SPARQL).")
                continue
                
            if "prefix" not in ground_truth_sparql.lower():
                ground_truth_sparql = BRICK_PREFIXES + ground_truth_sparql

            print(f"\n--- Processing Query Group ID: {query_id} ---")
            
            for question_obj in query_info.get('questions', []):
                question_text = question_obj.get('text')
                if not question_text:
                    continue

                if check_if_question_exists(question_text, LOG_FILE, MODEL_NAME):
                    print(f"Question '{question_text[:50]}...' already logged. Skipping.")
                    continue

                eval_data = {
                    'query_id': query_id,
                    'question_number': question_obj.get('question_number', 'N/A'),
                    'source': question_obj.get('source', 'N/A'),
                    'question': question_text,
                    'ground_truth_sparql': ground_truth_sparql
                }
                
                agent.refine_and_evaluate_query(
                    eval_data=eval_data, 
                    logger=logger, 
                    prefixes=BRICK_PREFIXES,
                    knowledge_graph_content=KNOWLEDGE_GRAPH_CONTENT
                )
                
                # --- Set flag and break after the first processed question ---
                print("\nFirst question processed. Exiting loops.")
                first_question_processed = True
                break # Exit the inner (questions) loop
            # --- Check the flag to exit the outer loop as well ---
            if first_question_processed:
                break # Exit the outer (queries) loop

    finally:
        logger.close()
        print("Logger closed.")

###  5.Configure the necessary test conditions: 
Set key parameters such as the target building (`BUILDING_NAME`), LLM (`MODEL_NAME`), and SPARQL endpoint.


In [6]:

MODEL_NAME = "openai/o3-mini"
BUILDING_NAME = "bldg11"
num_triples = 100
USE_LOCAL_TTL_FILE = False # We are using GraphDB for running queries. Convert this to True if you want to run queries locally.
LOCAL_TTL_PATH = f"./eval_buildings/{BUILDING_NAME}.ttl"
REMOTE_ENDPOINT_URL = f"http://Ozans-MacBook-Pro-9.local:7200/repositories/{BUILDING_NAME}" 
json_file_path = f"./Benchmark_QA_pairs/{BUILDING_NAME}_combined.json"
LOG_FILE = f"Example_Results/ReAct(w{num_triples})_{BUILDING_NAME}.csv"

run_single_question()


✅ Successfully extracted 23 prefixes from bldg11.ttl.
🔎 Original graph 'bldg11.ttl' contains 62578 triples.
   -> Graph is larger than 100 triples. Creating a subset for the prompt...
   -> ✅ Successfully created subset context with 100 triples.
First 1000 chars of knowledge graph content:
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix ns2: <http://buildsys.org/ontologies/bldg11#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix qudtqk: <http://qudt.org/vocab/quantitykind/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rec: <https://w3id.org/rec#> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix tag: <https://brickschema.org/schema/BrickTag#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns2:AHU01 brick:isFedBy ns2:chiller .

ns2:RM3530B a brick:HVAC_Zone

### 6. Process All Questions and Buildings

In [ ]:
def run_all_buildings(model_name: str, num_triples: int):
    # --- Configure your details here ---
    for BUILDING_NAME in ["b59", "bldg11","TUC_building","dflexlibs_multizone"]:  
        MODEL_NAME = model_name
        # The script will automatically detect if the target is a file path or a URL
        SPARQL_TARGET = LOCAL_TTL_PATH if USE_LOCAL_TTL_FILE else REMOTE_ENDPOINT_URL

        LOG_FILE = f"Results/ReAct(w{num_triples})_{BUILDING_NAME}.csv"

        # --- Dynamically get prefixes from the building's TTL file ---
        # This now correctly points to the local ttl file regardless of the query target.
        BRICK_PREFIXES = extract_prefixes_from_ttl(LOCAL_TTL_PATH)
        if not BRICK_PREFIXES:
            print("Could not extract prefixes. Exiting.")
            exit()
        
        KNOWLEDGE_GRAPH_CONTENT = get_kg_subset_content(LOCAL_TTL_PATH, max_triples= num_triples)
        print("First 1000 chars of knowledge graph content:")
        print(KNOWLEDGE_GRAPH_CONTENT[:1000])

        
        try:
            with open(json_file_path, 'r', encoding='utf-8') as f:
                all_data = json.load(f)
        except FileNotFoundError:
            print(f"❌ Error: The file '{json_file_path}' was not found.")
            exit()
        except json.JSONDecodeError:
            print(f"❌ Error: The file '{json_file_path}' is not a valid JSON file.")
            exit()

        if not isinstance(all_data, list) or not all_data:
            print(f"❌ Error: Expected JSON to be a non-empty list of building objects.")
            exit()
        target_building_data = all_data[0]

        #if log file dont exist, mkdir
        if not os.path.exists(os.path.dirname(LOG_FILE)):
            os.makedirs(os.path.dirname(LOG_FILE))


        logger = CsvLogger(filename=LOG_FILE, fieldnames=LOG_FIELDNAMES)
        agent = SparqlRefinementAgent(
            sparql_endpoint=SPARQL_TARGET, 
            model_name=MODEL_NAME, 
            max_iterations=3
        )

        print(f"--- Processing building: {BUILDING_NAME} for model: {MODEL_NAME} ---")
        
        try:
            for query_info in target_building_data.get('queries', []):
                query_id = query_info.get('query_id')
                ground_truth_sparql = query_info.get('sparql_query')

                if not ground_truth_sparql:
                    print(f"⚠️ Warning: Skipping Query Group ID {query_id} (no ground truth SPARQL).")
                    continue
                    
                if "prefix" not in ground_truth_sparql.lower():
                    ground_truth_sparql = BRICK_PREFIXES + ground_truth_sparql

                print(f"\n--- Processing Query Group ID: {query_id} ---")
                
                for question_obj in query_info.get('questions', []):
                    question_text = question_obj.get('text')
                    if not question_text:
                        continue

                    if check_if_question_exists(question_text, LOG_FILE, MODEL_NAME):
                        continue

                    eval_data = {
                        'query_id': query_id,
                        'question_number': question_obj.get('question_number', 'N/A'),
                        'source': question_obj.get('source', 'N/A'),
                        'question': question_text,
                        'ground_truth_sparql': ground_truth_sparql
                    }
                    
                    agent.refine_and_evaluate_query(
                        eval_data=eval_data, 
                        logger=logger, 
                        prefixes=BRICK_PREFIXES,
                        knowledge_graph_content=KNOWLEDGE_GRAPH_CONTENT
                    )
        
        finally:
            logger.close()


for num_triples in [100, 5000]:
    run_all_buildings(model_name="openai/o3-mini", num_triples=num_triples)